# Issuers' Logo

In [1]:
import numpy as np
import pandas as pd
import requests
import os
import time
import json

### Clean up data

In [8]:
issuers = pd.read_csv("2015-Q3-HIOS-Issuers.csv")

In [9]:
issuers.head(10)

,HIOS_ISSUER_ID,ISSR_LGL_NAME,MarketingName,State,IndividualMarket,SmallGroupMarket,UnknownMarket,LargeMarket,FEDERAL_EIN,Active,DateCreated,LastModifiedDate,DatabaseCompanyID,ORG_ADR1,ORG_ADR2,ORG_CITY,ORG_STATE,ORG_ZIP,ORG_ZIP4
0,97624,PreferredOne Community Health Plan,NaN,MN,YES,YES,NO,NO,411796007.0,YES,7/14/10,5/17/13,315.0,6105 Golden Hills Drive,NaN,Golden Valley,MN,55416,NaN
1,88102,PreferredOne Insurance Company,NaN,MN,YES,YES,NO,NO,810625135.0,YES,7/14/10,5/17/13,316.0,6501 Golden Hills Drive,NaN,Golden Valley,MN,55416,NaN
2,21580,"nHealth, Inc.",NaN,VA,NO,YES,NO,NO,760827573.0,YES,6/16/10,NaN,282.0,2570 Technical Drive,NaN,Miamisburg,OH,45342,NaN
3,18196,"Trilogy Health Insurance, Inc.",NaN,WI,NO,YES,NO,NO,205330218.0,YES,7/14/10,8/28/12,373.0,18000 West Sarah Lane,Suite 310,Brookfield,WI,53045,5842
4,47342,Health Tradition Health Plan,Health Tradition Health Plan,WI,YES,YES,NO,NO,391545987.0,YES,5/26/10,11/6/13,188.0,1808 E. Main St,NaN,Onalaska,WI,54650,NaN
5,47342,Health Tradition Health Plan,Health Tradition Health Plan,WI,YES,YES,NO,NO,391545987.0,YES,5/26/10,11/6/13,188.0,1808 East Main Street,PO BOX 188,Onalaska,WI,54650,8456
6,99852,"The Vermont Health Plan, LLC",NaN,VT,NO,YES,NO,NO,30354356.0,YES,5/21/10,3/8/13,369.0,P.O. Box 186,NaN,Montpelier,VT,5601,186
7,85425,Mutual of Omaha Insurance Company,NaN,AL,YES,NO,NO,NO,470246511.0,YES,6/11/10,NaN,269.0,Mutual of Omaha Plaza,NaN,Omaha,NE,68175,NaN
8,31204,Mutual of Omaha Insurance Company,NaN,MA,YES,NO,NO,NO,470246511.0,YES,6/11/10,NaN,269.0,Mutual of Omaha Plaza,NaN,Omaha,NE,68175,NaN
9,48149,Mutual of Omaha Insurance Company,NaN,NY,YES,NO,NO,NO,470246511.0,YES,6/11/10,NaN,269.0,Mutual of Omaha Plaza,NaN,Omaha,NE,68175,NaN


In [6]:
issuers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7417 entries, 0 to 7416
Data columns (total 19 columns):
HIOS_ISSUER_ID       7417 non-null int64
ISSR_LGL_NAME        7417 non-null object
MarketingName        2734 non-null object
State                7417 non-null object
IndividualMarket     7417 non-null object
SmallGroupMarket     7417 non-null object
UnknownMarket        7417 non-null object
LargeMarket          7417 non-null object
FEDERAL_EIN          7415 non-null float64
Active               7417 non-null object
DateCreated          7417 non-null object
LastModifiedDate     2505 non-null object
DatabaseCompanyID    7415 non-null float64
ORG_ADR1             7111 non-null object
ORG_ADR2             1613 non-null object
ORG_CITY             7116 non-null object
ORG_STATE            7162 non-null object
ORG_ZIP              7163 non-null object
ORG_ZIP4             3357 non-null object
dtypes: float64(2), int64(1), object(16)
memory usage: 1.1+ MB


In [10]:
# Get name column
issuers["name"] = np.where(pd.isnull(issuers['MarketingName']), issuers['ISSR_LGL_NAME'], issuers['MarketingName'])

In [11]:
# Parse the name columns
issuers['name_parsed'] = issuers['name'].str.lower()\
    .str.split(',|\(|\.').str.get(0).str.strip()\
    .str.split('(co|llc|inc)$').str.get(0).str.strip()

In [12]:
# Check names
issuers_names_array = list(set(issuers['name_parsed']))
print len(issuers_names_array)
print issuers_names_array

1309
['florida blue', 'thrivent financial for lutherans', "martin's point generations", 'nea ret health trust', 'premera blue cross', 'dsm usa insurance company- il', 'allegiance life & health insurance company', 'guarantee trust life insurance', 'american medical security life insurance company', 'north shore-lij insurance company', 'ffe rr test issuer m 524', 'conseco life insurance company', 'southwest life & health insurance', 'cigna healthcare of delaware', 'cgi test issuer 3', 'cgi test issuer 2', 'cgi djr issuer', 'healthplus insurance company', 'cgi test issuer 7', 'cgi test issuer 6', 'cgi test issuer 5', 'cgi test issuer 4', 'shelter life insurance company', 'unitedhealthcare insurance company', 'colorado choice health plans', 'north america administrators', 'humana health insurance co of fl', 'blue cross blue shield of louisiana', 'altus dental insurance company', 'golden dental plans', 'empire today', 'roquette america', 'sierra health and life insurance company', 'humana w

### Get images

In [13]:
def get_bing_images(query):
    bing_endpoint = "https://api.cognitive.microsoft.com/bing/v5.0/images/search"
    headers = {    
        'Ocp-Apim-Subscription-Key': os.environ.get("MSFT_KEY")
    }
    params = {
        'q': query,
        'count': '10',
        'offset': '0',
        'mkt': 'en-us',
        'safeSearch': 'Moderate',
    }
    return requests.get(url = bing_endpoint, params = params, headers = headers)

In [44]:
def parse_response(response):
    response_dict = response.json()
    return [ hit['contentUrl'] for hit in response_dict['value'] ]        

In [83]:
def process_names(names_array, file_path):
    with open(file_path, 'a') as f:
        for name in names_array:
            bing_response = get_bing_images(name + ' logo')
            time.sleep(0.2)
            f.write(json.dumps([name] + parse_response(bing_response)) + '\n')

#### Split Data Into Half 

In [76]:
names_array = list(issuers_names_array)

In [77]:
len(names_array)

1309

In [78]:
names_array_1, names_array_2 = names_array[:650], names_array[650:]

#### Process First Half

In [96]:
process_names(names_array_1, "image_results_1.txt")

In [14]:
!head -n 10 image_results_1.txt

["florida blue", "http://jacksonville.com/sites/default/files/FloridaBlue990.jpg", "http://www.infiniagroup.com/sites/default/files/Infinia_FloridaBlue_Logo.jpeg?1386971039", "http://jacksonville.com/sites/default/files/imagecache/superphoto/FloridaBlueLogo.jpg", "http://www.brandsoftheworld.com/sites/default/files/styles/logo-thumbnail/public/112012/florida_blue.png", "http://www.prestigeinsurancegrp.com/img/~www.prestigeinsurancegrp.com/florida_blue_Logo.jpg", "http://painmgmt.floridablue.com/sites/painmgmt/files/logo_FLB.png", "http://www.proplayerinsurancegroup.com/img/~www.Proplayerinsurancegroup.com/logo-florida-blue.jpg", "http://wochamber.com/images/uploads/trustee_logos/FLorida_Blue_Logo_-_For_Web.jpg", "http://a3.img.talkingpointsmemo.com/image/upload/c_fill,fl_keep_iptc,g_faces,h_365,w_652/blue-cross-and-blue-shield-of-florida-logo--2.jpg", "http://mountdoracenterforthearts.org/wp-content/uploads/2012/07/floridabcbs_logo.jpg"]
["thrivent financial for lutherans", "http://2.b

#### Process second half

In [100]:
process_names(names_array_2, "image_results_2.txt")

In [15]:
!head "image_results_2.txt"

["protective life corporation", "http://bloximages.newyork1.vip.townnews.com/stltoday.com/content/tncms/assets/v3/editorial/0/c2/0c216363-5a62-5e32-a893-8c4302ea46c2/538fd48c023cd.preview-620.jpg", "http://upload.wikimedia.org/wikipedia/en/2/24/Protectivelifelogo2.png", "http://newsroom.protective.com/sites/protective.newshq.businesswire.com/files/logo/image/Corp_Tag_Color.jpg", "http://slaughtergroup.com/files/8113/4642/2146/protective_brandmarks_02.jpg", "https://www.redrockinsurancegroup.com/images/carriers/life/14_protective.gif", "https://fbcdn-profile-a.akamaihd.net/hprofile-ak-xpa1/v/t1.0-1/c3.0.160.160/p160x160/1256526_10151867927416458_1725804594_n.jpg?oh=605e4374524ca8cf73fc49a39e6e8840&oe=563AE5F3&__gda__=1447590169_bc683001005f4e8912f59a10dc0b0e0e", "http://www.trademarkia.com/logo-images/protective-life-insurance-company/inflation-shield-85501976.jpg", "http://upload.wikimedia.org/wikipedia/en/thumb/4/45/UIL_stacked_logo.JPG/200px-UIL_stacked_logo.JPG", "http://logosandbra

### Merge back into original df

In [102]:
!cat image_results* > image_results_total.txt

In [103]:
!wc -l image_results_total.txt

    1309 image_results_total.txt


In [104]:
def read_image_urls(file_path):
    url_array = []
    with open(file_path, 'r') as f:
        for line in f:
            url_array.append(json.loads(line.strip()))
    # Turn into dataframe
    column_names = ['name_parsed'] + ['image_url_' + str(i+1) for i in range(10)]    
    return pd.DataFrame(url_array, columns = column_names)

In [105]:
df_image_urls = read_image_urls("image_results_total.txt")

In [106]:
df_image_urls.head()

,name_parsed,image_url_1,image_url_2,image_url_3,image_url_4,image_url_5,image_url_6,image_url_7,image_url_8,image_url_9,image_url_10
0,florida blue,http://jacksonville.com/sites/default/files/Fl...,http://www.infiniagroup.com/sites/default/file...,http://jacksonville.com/sites/default/files/im...,http://www.brandsoftheworld.com/sites/default/...,http://www.prestigeinsurancegrp.com/img/~www.p...,http://painmgmt.floridablue.com/sites/painmgmt...,http://www.proplayerinsurancegroup.com/img/~ww...,http://wochamber.com/images/uploads/trustee_lo...,http://a3.img.talkingpointsmemo.com/image/uplo...,http://mountdoracenterforthearts.org/wp-conten...
1,thrivent financial for lutherans,http://2.bp.blogspot.com/_DmtdGP6kzMQ/S7jkzluN...,http://stmedia.startribune.com/images/ows_1393...,http://ogpdn1wn2d93vut8u40tokx1dl7.wpengine.ne...,http://1.bp.blogspot.com/-_KZnl3VHs4c/UyLuKKH1...,https://www.thrivent.com/images/logo_thrivent_...,http://www.grandforksherald.com/sites/default/...,http://www.logotypes101.com/logos/757/2FD40374...,http://s3.evcdn.com/images/edpborder500/I0-001...,http://www.lss-elca.org/Redesign/wp-content/up...,http://www.lhfmissions.org/image/4---resources...
2,martin's point generations,https://martinspoint.org/about-martins-point/~...,http://robbiefoundation.com/rf/wp-content/uplo...,http://www.mainequalitycounts.org/image_upload...,http://rmbvivid.com/wp-content/uploads/2013/01...,http://www.trademarkia.com/logo-images/martins...,http://www.citbni.org.uk/getfile/8cdb9c32-823b...,http://seeklogo.com/images/N/New_Generation_Po...,http://pointsolutionsuk.com/wp-content/uploads...,http://www.stillpointfarmdressage.com/logo%20m...,http://www.pycomall.com/images/P1/Remy_Martin_...
3,nea ret health trust,http://www.amh.org.uk/wp-content/uploads/2013/...,http://www.singforyourlife.org.uk/sites/defaul...,http://www.enhancefitstudio.com/wp-content/upl...,http://862886595.r.lightningbase-cdn.com/wp-co...,http://upload.wikimedia.org/wikipedia/commons/...,https://www.universityparent.com/wp-content/up...,http://www.health.qld.gov.au/pahospital/resear...,http://upload.wikimedia.org/wikipedia/commons/...,http://www.sexualhealthhounslow.org.uk/images/...,http://www.health.qld.gov.au/pahospital/resear...
4,premera blue cross,http://www.wtae.com/image/view/-/31860568/medR...,https://www.kintera.org/AccountTempFiles/accou...,http://assets.nydailynews.com/polopoly_fs/1.21...,https://pbs.twimg.com/profile_images/461229740...,http://www.trademarkia.com/logo-images/premera...,http://www.kingcountyhealthcarecoalition.org/m...,https://payments.bankofamerica.com/epaybill/im...,https://www.premeralawsuit.com/wp-content/uplo...,http://www.distilnfo.com/payer/files/2013/09/7...,http://snocohealth.org/wp-content/uploads/2013...


In [108]:
issuers_merged = issuers.merge(df_image_urls, how='left', on='name_parsed')

In [109]:
issuers_merged.head()

,HIOS_ISSUER_ID,ISSR_LGL_NAME,MarketingName,State,IndividualMarket,SmallGroupMarket,UnknownMarket,LargeMarket,FEDERAL_EIN,Active,...,image_url_1,image_url_2,image_url_3,image_url_4,image_url_5,image_url_6,image_url_7,image_url_8,image_url_9,image_url_10
0,97624,PreferredOne Community Health Plan,NaN,MN,YES,YES,NO,NO,411796007.0,YES,...,http://mncm.org/wp-content/uploads/2013/04/Pre...,http://www.health.state.mn.us/divs/hpsc/mcs/pr...,http://img.docstoccdn.com/thumb/orig/58158416.png,http://www.vvhc.org/images/chp-logo.gif,http://chpw.org/images/CHPW_logos/Healthfirst-...,http://vnacolorado.org/wp-content/uploads/2015...,http://healthnutsmedia.com/wp-content/uploads/...,http://www.communityplans.net/portals/0/Vendor...,http://www.lincolnhealthbuddy.co.uk/wp-content...,http://www.returningveterans.org/sites/default...
1,88102,PreferredOne Insurance Company,NaN,MN,YES,YES,NO,NO,810625135.0,YES,...,https://static.ehealthinsurance.com/ehealthins...,http://www.mnhealthinsurance.com/images/logos/...,http://www.mnhealthinsuranceagency.com/wp-cont...,http://www.healthinsurance-quotes.org/images/l...,https://m1.behance.net/rendition/modules/88593...,http://www.beckagencyinc.com/images/norwegianl...,http://m1.behance.net/rendition/modules/882654...,http://m1.behance.net/rendition/modules/882654...,http://m1.behance.net/rendition/modules/882654...,https://mnhealthnetwork.com/images/MagnumLogo_...
2,21580,"nHealth, Inc.",NaN,VA,NO,YES,NO,NO,760827573.0,YES,...,http://mega.startupweekend.org/files/2011/09/h...,http://www.ranklogos.com/wp-content/uploads/20...,http://www.ranklogos.com/wp-content/uploads/20...,http://www.ranklogos.com/wp-content/uploads/20...,http://www.ranklogos.com/wp-content/uploads/20...,http://2.bp.blogspot.com/-tOGcbF4LiNU/T6tr41Re...,http://www.ranklogos.com/wp-content/uploads/20...,http://ranklogos.com/wp-content/uploads/2012/0...,http://www.ranklogos.com/wp-content/uploads/20...,http://www.liebermanresearch.com/images/logos_...
3,18196,"Trilogy Health Insurance, Inc.",NaN,WI,NO,YES,NO,NO,205330218.0,YES,...,http://www.trilogyfs.com/App_Themes/Trilogy/im...,http://cdn2.hubspot.net/hub/29383/file-2477044...,http://www.trilogygroupbenefits.com/files/Imag...,http://www.commongroundwi.org/wp-content/uploa...,http://media.merchantcircle.com/41103543/Trilo...,http://www.davidraefp.com/files/24519/trilogy%...,http://www.theljbgroup.com/Content/Trilogy%20L...,http://www.viainsurance.com/Portals/29383/imag...,http://www.localfirstaz.com/aalogos/240wide/68...,http://www.viainsurance.com/Portals/29383/imag...
4,47342,Health Tradition Health Plan,Health Tradition Health Plan,WI,YES,YES,NO,NO,391545987.0,YES,...,http://medigapandyou.com/wp-content/plugins/ma...,http://www.express-press-release.net/74/logo/s...,http://medigapandyou.com/wp-content/plugins/ma...,http://ruralhc.net/media/ruralhcfactor360com/u...,http://www.myplanportal.com/ebusiness/assets/i...,http://iuhealth.cloudapp.net/images/site-image...,http://www.fitchburgchamber.com/wp-content/the...,http://www.infusionresource.com/media/tufts-he...,http://www.texaschildrenshealthplan.org/skinin...,http://cf.juggle-images.com/matte/white/280x28...


In [117]:
issuers_merged.to_csv("issuers_image_urls.csv", index = False, encoding = 'utf-8')

In [2]:
### Take image_url_1 column

In [2]:
df = pd.read_csv("issuers_image_urls.csv")

In [3]:
logos = df.loc[:, ['HIOS_ISSUER_ID', 'ISSR_LGL_NAME', 'MarketingName', 'State', 'image_url_1']].groupby('HIOS_ISSUER_ID').first()

In [4]:
logos.head()

,ISSR_LGL_NAME,MarketingName,State,image_url_1
HIOS_ISSUER_ID,,,,
10011,Southern Farm Bureau Life Insurance Company,NaN,TX,http://www.insurancelevel.com/company/logo/far...
10018,Unified Life Insurance Company,NaN,SC,http://www.texaskidsfirst.com/images/ULIC-Logo...
10029,American National Life Insurance Company of Texas,NaN,MD,http://www.moodyinsurancegroup.com/wp-content/...
10046,BEST Life and Health Insurance Company,BEST Life,HI,http://hersheys.newslinevine.com/images/full/b...
10047,GTECH Corporatopn,GTECH Corporation,RI,http://caribbeanresumes.com/files/imagecache/e...


In [5]:
logos.to_csv("issuers_logos.csv", index = True, encoding = 'utf-8')